Names.csv 
* Dodaj kolumnę z wartością czasu wykonania notatnika w formacie epoch
* Dodaj kolumnę w której wyliczysz wzrost w stopach (feet)
* Odpowiedz na pytanie jakie jest najpopularniesze imię?
* Dodaj kolumnę i policz wiek aktorów 
* Usuń kolumny (bio, death_details)
* Zmień nazwy kolumn - dodaj kapitalizaję i usuń _
* Posortuj dataframe po imieniu rosnąco

In [0]:
from pyspark.sql.functions import *

filePath = "dbfs:/FileStore/tables/Files/names.csv"
namesDf = spark.read.format("csv")\
              .option("header", "true")\
              .option("inferSchema", "true")\
              .load(filePath)

# Dodaj kolumnę z wartością czasu wykonania notatnika w formacie epoch
namesDf = namesDf.withColumn("Time", unix_timestamp())

# Dodaj kolumnę w której wyliczysz wzrost w stopach (feet)
namesDf = namesDf.withColumn("HeightFeet", round(col("height") / 30.48, 2))

# Odpowiedz na pytanie jakie jest najpopularniesze imię?
common_name = namesDf.groupBy("name").count().orderBy(col("count").desc()).select("name").first()[0]
print(f"Najpopularniejsze imię: {common_name}")

namesDf = namesDf.withColumn("date_of_birth", regexp_extract(col("date_of_birth"), r"(\d{4})", 1).cast("int"))
namesDf = namesDf.withColumn("date_of_death", regexp_extract(col("date_of_death"), r"(\d{4})", 1).cast("int"))

# Dodaj kolumnę i policz wiek aktorów
namesDf = namesDf.withColumn("ActorAge", 
    when(col("date_of_birth").isNotNull(),
         when(col("date_of_death").isNotNull(), col("date_of_death") - col("date_of_birth"))
         .otherwise(year(current_date()) - col("date_of_birth"))
    )
)

# Usuń kolumny (bio, death_details)
namesDf = namesDf.drop("bio", "death_details")

# Zmień nazwy kolumn - dodaj kapitalizaję i usuń _
namesDf = namesDf.select([col(c).alias(c.replace("_", " ").title()) for c in namesDf.columns])

# Posortuj dataframe po imieniu rosnąco
namesDf = namesDf.orderBy("Name")

display(namesDf.limit(10))


Najpopularniejsze imię: David Campbell


Imdb Name Id,Name,Birth Name,Height,Birth Details,Date Of Birth,Place Of Birth,Date Of Death,Place Of Death,Reason Of Death,Spouses String,Spouses,Divorces,Spouses With Children,Children,Time,Heightfeet,Actorage
nm1001478,'Big' LeRoy Mobley,LeRoy King Mobley III,193,"April 1, 1973 in Atlantic City, New Jersey, USA",1973,"Atlantic City, New Jersey, USA",null,null,null,null,0,0,0,0,1742946019,6.33,52
nm0521811,'Ducky' Louie,Lawrence Louie,null,"July 22, 1931 in Berkeley, California, USA",1931,"Berkeley, California, USA",null,null,null,null,0,0,0,0,1742946019,null,94
nm0722372,'Little Billy' Rhodes,William H. Rhodes,null,"February 1, 1895 in Illinois, USA",1895,"Illinois, USA",1967,"Hollywood, California, USA",stroke,null,0,0,0,0,1742946019,null,72
nm0946148,'Weird Al' Yankovic,Alfred Matthew Yankovic,183,"October 23, 1959 in Downey, California, USA",1959,"Downey, California, USA",null,null,null,Suzanne Krajewski (10 February 2001 - present) (1 child),1,0,1,1,1742946019,6.0,66
nm1265067,50 Cent,Curtis James Jackson III,183,"July 6, 1975 in Queens, New York City, New York, USA",1975,"Queens, New York City, New York, USA",null,null,null,null,0,0,0,0,1742946019,6.0,50
nm0553436,A Martinez,Adolph Larrue Martinez III,175,"September 27, 1948 in Glendale, California, USA",1948,"Glendale, California, USA",null,null,null,Leslie Bryans (17 July 1982 - present) (3 children)Mare Winningham (1981 - 29 January 1982) (divorced),2,1,1,3,1742946019,5.74,77
nm1100197,A. Baldwin Sloane,A. Baldwin Sloane,null,"August 28, 1872 in Baltimore, Maryland, USA",1872,"Baltimore, Maryland, USA",1925,"Red Bank, New Jersey, USA",null,null,0,0,0,0,1742946019,null,53
nm0080406,A. Bhimsingh,A. Bhimsingh,null,"July 15, 1924 in Tirupati, Andhra Pradesh, India",1924,"Tirupati, Andhra Pradesh, India",1978,"Madras, Tamil Nadu, India",null,Sukumari (? - 16 January 1978) (his death) (1 child),1,0,1,1,1742946019,null,54
nm0770661,A. Hans Scheirl,Angela Hans Schierl,null,"1956 in Salzburg, Austria",1956,"Salzburg, Austria",null,null,null,null,0,0,0,0,1742946019,null,69
nm0072200,A. Jonathan Benny,A. Jonathan Benny,null,"November 4, 1970",1970,null,null,null,null,null,0,0,0,0,1742946019,null,55


Movies.csv
* Dodaj kolumnę z wartością czasu wykonania notatnika w formacie epoch
* Dodaj kolumnę która wylicza ile lat upłynęło od publikacji filmu
* Dodaj kolumnę która pokaże budżet filmu jako wartość numeryczną, (trzeba usunac znaki walut)
* Usuń wiersze z dataframe gdzie wartości są null

In [0]:
from pyspark.sql.functions import *

filePath = "dbfs:/FileStore/tables/Files/movies.csv"
moviesDf = spark.read.format("csv")\
    .option("header", "true")\
    .option("inferSchema", "true")\
    .load(filePath)

# Dodanie kolumny z czasem w formacie epoch
moviesDf = moviesDf.withColumn("Time", unix_timestamp())

# Ekstrakcja roku z kolumny date_published (jeśli występują inne znaki, usuwamy je)
moviesDf = moviesDf.withColumn("date_published", regexp_extract(col("date_published"), r"(\d{4})", 1).cast("int"))

# Obliczanie lat od publikacji filmu
moviesDf = moviesDf.withColumn("YearsSinceRelease", year(current_date()) - col("date_published"))

# Konwersja budżetu na liczbę (usunięcie znaków walut)
moviesDf = moviesDf.withColumn("BudgetNumeric", regexp_extract(col("budget"), r"(\d+)", 1).cast("int"))

# Usunięcie wierszy z brakującymi wartościami
moviesDf = moviesDf.dropna()

display(moviesDf.limit(10))



imdb_title_id,title,original_title,year,date_published,genre,duration,country,language,director,writer,production_company,actors,description,avg_vote,votes,budget,usa_gross_income,worlwide_gross_income,metascore,reviews_from_users,reviews_from_critics,Time,YearsSinceRelease,BudgetNumeric
tt0017136,Metropolis,Metropolis,1927,1928,"Drama, Sci-Fi",153,Germany,German,Fritz Lang,"Thea von Harbou, Thea von Harbou",Universum Film (UFA),"Alfred Abel, Gustav Fröhlich, Rudolf Klein-Rogge, Fritz Rasp, Theodor Loos, Erwin Biswanger, Heinrich George, Brigitte Helm","In a futuristic city sharply divided between the working class and the city planners, the son of the city's mastermind falls in love with a working class prophet who predicts the coming of a savior to mediate their differences.",08.mar,156076,DEM 6000000,$ 1236166,$ 1349711,98.0,495.0,208.0,1742945919,97,6000000
tt0021749,Luci della città,City Lights,1931,1931,"Comedy, Drama, Romance",87,USA,English,Charles Chaplin,Charles Chaplin,Charles Chaplin Productions,"Virginia Cherrill, Florence Lee, Harry Myers, Al Ernest Garcia, Hank Mann, Charles Chaplin","With the aid of a wealthy erratic tippler, a dewy-eyed tramp who has fallen in love with a sightless flower girl accumulates money to be able to help her medically.",08.maj,162668,$ 1500000,$ 19181,$ 46008,99.0,295.0,122.0,1742945919,94,1500000
tt0027977,Tempi moderni,Modern Times,1936,1937,"Comedy, Drama, Family",87,USA,English,Charles Chaplin,Charles Chaplin,Charles Chaplin Productions,"Charles Chaplin, Paulette Goddard, Henry Bergman, Tiny Sandford, Chester Conklin, Hank Mann, Stanley Blystone, Al Ernest Garcia, Richard Alexander, Cecil Reynolds, Mira McKinney, Murdock MacQuarrie, Wilfred Lucas, Edward LeSaint, Fred Malatesta",The Tramp struggles to live in modern industrial society with the help of a young homeless woman.,08.maj,211250,$ 1500000,$ 163577,$ 457688,96.0,307.0,115.0,1742945919,88,1500000
tt0029453,Il bandito della Casbah,Pépé le Moko,1937,1937,"Crime, Drama, Romance",94,France,"French, Arabic",Julien Duvivier,"Henri La Barthe, Henri La Barthe",Paris Film,"Jean Gabin, Gabriel Gabrio, Saturnin Fabre, Fernand Charpin, Lucas Gridoux, Gilbert Gil, Marcel Dalio, Charles Granval, Gaston Modot, René Bergeron, Paul Escoffier, Roger Legris, Jean Témerson, Robert Ozanne, Philippe Richard","A wanted gangster is both king and prisoner of the Casbah. He is protected from arrest by his friends, but is torn by his desire for freedom outside. A visiting Parisian beauty may just tempt his fate.",07.lip,6180,$ 60000,$ 155895,$ 155895,98.0,46.0,55.0,1742945919,88,60000
tt0029583,Biancaneve e i sette nani,Snow White and the Seven Dwarfs,1937,1938,"Animation, Family, Fantasy",83,USA,English,"William Cottrell, David Hand","Jacob Grimm, Wilhelm Grimm",Walt Disney Productions,"Roy Atwell, Stuart Buchanan, Adriana Caselotti, Eddie Collins, Pinto Colvig, Marion Darlington, Billy Gilbert, Otis Harlan, Lucille La Verne, James MacDonald, Scotty Mattraw, Moroni Olsen, Purv Pullen, Harry Stockwell, Bill Thompson","Exiled into the dangerous forest by her wicked stepmother, a princess is rescued by seven dwarf miners who make her part of their household.",07.cze,177157,$ 1499000,$ 184925486,$ 184925486,95.0,260.0,173.0,1742945919,87,1499000
tt0031381,Via col vento,Gone with the Wind,1939,1949,"Drama, History, Romance",238,USA,English,"Victor Fleming, George Cukor","Margaret Mitchell, Sidney Howard",Selznick International Pictures,"Thomas Mitchell, Barbara O'Neil, Vivien Leigh, Evelyn Keyes, Ann Rutherford, George Reeves, Fred Crane, Hattie McDaniel, Oscar Polk, Butterfly McQueen, Victor Jory, Everett Brown, Howard Hickman, Alicia Rhett, Leslie Howard",A manipulative woman and a roguish man conduct a turbulent romance during the American Civil War and Reconstruction periods.,08.sty,283975,$ 3977000,$ 200852579,$ 402352579,97.0,881.0,197.0,1742945919,76,3977000
tt0031679,Mr. Smith va a Washington,Mr. Smith Goes to Washington,1939,1947,"Comedy, Drama

ratings.csv
* Dodaj kolumnę z wartością czasu wykonania notatnika w formacie epoch
* Dla każdego z poniższych wyliczeń nie bierz pod uwagę `nulls` 
* Kto daje lepsze oceny chłopcy czy dziewczyny dla całego setu
* Dla jednej z kolumn zmień typ danych do `long` 

In [0]:
from pyspark.sql.functions import *

filePath = "dbfs:/FileStore/tables/Files/ratings.csv"
ratingsDf = spark.read.format("csv")\
              .option("header","true")\
              .option("inferSchema","true")\
              .load(filePath)

# Dodaj kolumnę z wartością czasu wykonania notatnika w formacie epoch
ratingsDf = ratingsDf.withColumn("Time", unix_timestamp())

# Dla każdego z poniższych wyliczeń nie bierz pod uwagę nulls
ratingsDf = ratingsDf.dropna()

# Dla jednej z kolumn zmień typ danych do long
ratingsDf = ratingsDf.withColumn("mean_vote", col("mean_vote").cast("long"))

# Oblicz średnie oceny dla mężczyzn i kobiet
avg_male_rating = ratingsDf.select(avg("males_allages_avg_vote")).collect()[0][0]
avg_female_rating = ratingsDf.select(avg("females_allages_avg_vote")).collect()[0][0]

# Sprawdzenie, kto daje lepsze oceny
gender = "mężczyźni" if avg_male_rating > avg_female_rating else "kobiety"
print(f"Lepsze oceny wystawiają: {gender}")

display(ratingsDf.limit(10))

Lepsze oceny wystawiają: kobiety


imdb_title_id,weighted_average_vote,total_votes,mean_vote,median_vote,votes_10,votes_9,votes_8,votes_7,votes_6,votes_5,votes_4,votes_3,votes_2,votes_1,allgenders_0age_avg_vote,allgenders_0age_votes,allgenders_18age_avg_vote,allgenders_18age_votes,allgenders_30age_avg_vote,allgenders_30age_votes,allgenders_45age_avg_vote,allgenders_45age_votes,males_allages_avg_vote,males_allages_votes,males_0age_avg_vote,males_0age_votes,males_18age_avg_vote,males_18age_votes,males_30age_avg_vote,males_30age_votes,males_45age_avg_vote,males_45age_votes,females_allages_avg_vote,females_allages_votes,females_0age_avg_vote,females_0age_votes,females_18age_avg_vote,females_18age_votes,females_30age_avg_vote,females_30age_votes,females_45age_avg_vote,females_45age_votes,top1000_voters_rating,top1000_voters_votes,us_voters_rating,us_voters_votes,non_us_voters_rating,non_us_voters_votes,Time
tt0000009,5.9,154,5,6.0,12,4,10,43,28,28,9,1,5,14,7.2,4.0,6.0,38.0,5.7,50.0,6.6,35.0,6.2,97.0,7.0,1.0,5.9,24.0,5.6,36.0,6.7,31.0,6.0,35.0,7.3,3.0,5.9,14.0,5.7,13.0,4.5,4.0,5.7,34.0,6.4,51.0,6.0,70.0,1742945999
tt0002130,7.0,2237,6,7.0,210,225,436,641,344,169,66,39,20,87,7.5,4.0,7.0,402.0,7.0,895.0,7.1,482.0,7.0,1607.0,8.0,2.0,7.0,346.0,7.0,804.0,7.0,396.0,7.2,215.0,7.0,2.0,7.0,52.0,7.3,82.0,7.4,77.0,6.9,139.0,7.0,488.0,7.0,1166.0,1742945999
tt0003740,7.1,3073,6,7.0,285,301,591,727,443,199,85,27,18,397,6.0,3.0,7.0,393.0,7.0,1126.0,7.2,1006.0,7.1,2149.0,6.0,2.0,7.0,323.0,7.0,976.0,7.3,799.0,6.9,402.0,6.0,1.0,6.8,67.0,6.9,134.0,6.8,194.0,6.9,177.0,7.0,1035.0,7.0,1332.0,1742945999
tt0004972,6.3,22213,6,7.0,3661,1741,3314,3963,2876,1928,978,701,577,2474,4.7,14.0,6.0,3183.0,6.3,8861.0,6.7,4901.0,6.4,14818.0,4.7,12.0,6.1,2623.0,6.4,7669.0,6.8,4104.0,5.7,2417.0,6.0,1.0,5.4,517.0,5.8,1106.0,5.9,725.0,6.5,355.0,6.3,7452.0,6.4,8306.0,1742945999
tt0005680,5.9,130,5,6.0,4,3,8,20,52,27,8,5,2,1,5.0,3.0,5.8,19.0,5.9,44.0,5.9,50.0,5.9,102.0,5.0,2.0,5.8,16.0,5.8,40.0,6.0,43.0,5.9,12.0,5.0,1.0,5.5,2.0,6.5,4.0,6.2,5.0,5.7,50.0,5.9,26.0,5.8,87.0,1742945999
tt0006206,7.3,4166,6,7.0,620,439,763,872,484,232,140,40,31,545,6.5,7.0,7.1,517.0,7.3,1661.0,7.3,1294.0,7.3,2835.0,6.8,6.0,7.1,414.0,7.3,1384.0,7.3,976.0,7.2,670.0,4.0,1.0,6.9,98.0,7.1,256.0,7.7,306.0,6.8,204.0,7.5,1350.0,7.2,1945.0,1742945999
tt0006864,7.8,13875,7,8.0,3477,2230,3214,2249,1179,605,340,181,133,267,7.9,8.0,7.8,1795.0,7.7,5451.0,7.8,3667.0,7.8,9441.0,8.0,7.0,7.9,1498.0,7.7,4734.0,7.8,2970.0,7.5,1632.0,7.0,1.0,7.2,276.0,7.4,660.0,8.0,662.0,7.5,321.0,7.7,4286.0,7.8,5954.0,1742945999
tt0009611,7.3,5895,6,7.0,552,564,1303,1742,724,228,69,35,22,656,5.0,4.0,7.3,833.0,7.3,2162.0,7.3,1730.0,7.3,4074.0,4.7,3.0,7.3,729.0,7.3,1888.0,7.3,1340.0,7.4,746.0,6.0,1.0,7.4,96.0,7.3,257.0,7.6,380.0,6.8,283.0,7.3,1734.0,7.3,2694.0,1742945999
tt0010323,8.1,55601,7,8.0,11426,11262,15971,8883,3517,1562,725,483,348,1424,8.5,47.0,8.1,10734.0,8.1,23386.0,8.0,8271.0,8.1,35834.0,8.5,38.0,8.1,8304.0,8.0,19321.0,8.0,6821.0,8.1,7660.0,8.8,7.0,8.1,2236.0,8.2,3807.0,8.1,1341.0,7.5,559.0,8.0,13136.0,8.1,25399.0,1742945999
tt0011130,7.0,4753,7,7.0,393,335,965,1666,816,326,107,54,35,56,7.0,4.0,6.9,546.0,7.0,1830.0,7.0,1428.0,7.0,3253.0,6.0,3.0,6.9,415.0,6.9,1541.0,7.0,1207.0,7.2,606.0,10.0,1.0,6.8,128.0,7.3,269.0,7.2,202.0,6.9,239.0,7.1,1733.0,6.9,1815.0,1742945999


# Zadanie 2

Spark UI to interfejs użytkownika dostępny w Apache Spark, który pozwala monitorować wykonywane zadania, optymalizować zapytania oraz analizować zużycie zasobów. Główne elementy Spark UI to:

- Jobs – pokazuje listę uruchomionych zadań (jobs), ich status oraz czas wykonania. Możemy tutaj zobaczyć, które zadania są w toku, zakończone lub niepowodzone.
- Stages – przedstawia etapy przetwarzania (stages) w ramach danego zadania. Pokazuje, które partycje danych są przetwarzane oraz jak Spark podzielił zapytanie na mniejsze kroki.
- Storage – zawiera informacje o przechowywanych zbiorach RDD i DataFrame w pamięci oraz na dysku, co pomaga w analizie zarządzania pamięcią.
- Environment – przedstawia konfigurację systemową Sparka, w tym wersję Javy, wersję Sparka oraz zmienne środowiskowe.
- Executors – wyświetla szczegółowe informacje o wykonawcach (executors), ich pamięci, liczbie zadań oraz wykorzystaniu CPU.
- SQL / Dataframe – pozwala przeanalizować zapytania SQL i operacje na DataFrame, w tym ich plany wykonania oraz optymalizacje.

# Zadanie 3

In [0]:
from pyspark.sql.functions import col, round

filePath = "dbfs:/FileStore/tables/Files/names.csv"
namesDf = spark.read.format("csv").option("header", "true")\
                    .option("inferSchema", "true")\
                    .load(filePath)

namesDf = namesDf.withColumn("height_in_feet", round(col("height") / 30.48, 2))

namesDf.explain()

namesGroup = namesDf.groupBy("divorces").count()

namesGroup.explain()

namesGroup.show()

== Physical Plan ==
*(1) Project [imdb_name_id#628, name#629, birth_name#630, height#631, bio#632, birth_details#633, date_of_birth#634, place_of_birth#635, death_details#636, date_of_death#637, place_of_death#638, reason_of_death#639, spouses_string#640, spouses#641, divorces#642, spouses_with_children#643, children#644, round((cast(height#631 as double) / 30.48), 2) AS height_in_feet#662]
+- FileScan csv [imdb_name_id#628,name#629,birth_name#630,height#631,bio#632,birth_details#633,date_of_birth#634,place_of_birth#635,death_details#636,date_of_death#637,place_of_death#638,reason_of_death#639,spouses_string#640,spouses#641,divorces#642,spouses_with_children#643,children#644] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex(1 paths)[dbfs:/FileStore/tables/Files/names.csv], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<imdb_name_id:string,name:string,birth_name:string,height:int,bio:string,birth_details:stri...


== Physical Plan ==
AdaptiveSparkPl